In [4]:
class Patient(object):
    # __init__ is a special method called whenever you try to make
    # an instance of a class. As you heard, it initializes the object.
    # Here, we'll initialize some of the data.
    def __init__(self):
        # Let's add some data to the [instance of the] class.
        self.age = 60
        self.timeontreatment = 0
        self.alive = True

    # We can also add our own functions. When our ball bounces,
    # its vertical velocity will be negated. (no gravity here!)
    def treat(self):
        self.timeontreatment += 1
        if self.timeontreatment > 28:
            self.alive = False


p0 = Patient()

# game loop
for i in range(5):
    p0.treat()

p0.timeontreatment, p0.alive

(5, True)

https://python-course.eu/applications-python/finite-state-machine.php

In [7]:
class StateMachine:
    
    def __init__(self):
        self.handlers = {}
        self.startState = None
        self.endStates = []

    def add_state(self, name, handler, end_state=0):
        name = name.upper()
        self.handlers[name] = handler
        if end_state:
            self.endStates.append(name)

    def set_start(self, name):
        self.startState = name.upper()

    def run(self, cargo):
        try:
            handler = self.handlers[self.startState]
        except:
            raise InitializationError("must call .set_start() before .run()")
        if not self.endStates:
            raise  InitializationError("at least one state must be an end_state")
    
        while True:
            (newState, cargo) = handler(cargo)
            if newState.upper() in self.endStates:
                print("reached ", newState)
                break 
            else:
                handler = self.handlers[newState.upper()] 

In [8]:
positive_adjectives = ["great","super", "fun", "entertaining", "easy"]
negative_adjectives = ["boring", "difficult", "ugly", "bad"]

def start_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word == "Python":
        newState = "Python_state"
    else:
        newState = "error_state"
    return (newState, txt)

def python_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word == "is":
        newState = "is_state"
    else:
        newState = "error_state"
    return (newState, txt)

def is_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word == "not":
        newState = "not_state"
    elif word in positive_adjectives:
        newState = "pos_state"
    elif word in negative_adjectives:
        newState = "neg_state"
    else:
        newState = "error_state"
    return (newState, txt)

def not_state_transitions(txt):
    splitted_txt = txt.split(None,1)
    word, txt = splitted_txt if len(splitted_txt) > 1 else (txt,"")
    if word in positive_adjectives:
        newState = "neg_state"
    elif word in negative_adjectives:
        newState = "pos_state"
    else:
        newState = "error_state"
    return (newState, txt)

def neg_state(txt):
    print("Hallo")
    return ("neg_state", "")


m = StateMachine()
m.add_state("Start", start_transitions)
m.add_state("Python_state", python_state_transitions)
m.add_state("is_state", is_state_transitions)
m.add_state("not_state", not_state_transitions)
m.add_state("neg_state", None, end_state=1)
m.add_state("pos_state", None, end_state=1)
m.add_state("error_state", None, end_state=1)
m.set_start("Start")
m.run("Python is great")
m.run("Python is difficult")
m.run("Perl is ugly")

reached  pos_state
reached  neg_state
reached  error_state


I've created a clean, extensible Python framework for modeling patient treatment flows as finite state machines. Here are the key features:

**Core Components:**
1. **Patient class** - Each patient is an FSM tracking their current state, time spent in each state, and complete history
2. **TransitionRule class** - Defines when/how patients move between states with configurable probabilities and conditions
3. **TreatmentFlowModel class** - Orchestrates the simulation and manages all patients

**Key Design Decisions:**
- **Flexible transitions**: Rules support minimum durations, probabilities, and custom conditions
- **Complete history tracking**: Every patient's journey is recorded for analysis
- **Easy data readout**: Built-in methods for state distributions, outcomes, journey details, and statistics
- **Extensibility**: Adding new states or transition rules is straightforward

**To Adapt This Framework:**
1. **Add states**: Extend the `PatientState` enum
2. **Add transition rules**: Create new `TransitionRule` objects with your logic
3. **Custom conditions**: Use the `condition` parameter for complex transition logic
4. **Add data readouts**: The model class makes it easy to add new analysis methods

The example simulates 100 patients through a 3-treatment pathway with various outcomes (remission or death). Run it to see summary statistics and individual patient journeys!

In [9]:
"""
Patient Treatment Flow Framework using Finite State Machines

This framework models patient journeys through treatment as FSMs, allowing 
easy tracking of states, transitions, and outcomes over time.
"""

from dataclasses import dataclass, field
from typing import Dict, List, Optional, Callable
from enum import Enum
import random


# ============================================================================
# STATE DEFINITIONS
# ============================================================================

class PatientState(Enum):
    """Define all possible patient states"""
    DIAGNOSED = "diagnosed"
    TREATMENT_1 = "treatment_1"
    TREATMENT_2 = "treatment_2"
    TREATMENT_3 = "treatment_3"
    REMISSION = "remission"
    DECEASED = "deceased"


# ============================================================================
# PATIENT CLASS (Finite State Machine)
# ============================================================================

@dataclass
class Patient:
    """
    Represents a patient as a finite state machine.
    Tracks current state, history, and time spent in each state.
    """
    id: int
    current_state: PatientState = PatientState.DIAGNOSED
    time_in_current_state: int = 0
    state_history: List[tuple] = field(default_factory=list)  # (state, duration)
    total_time: int = 0
    
    def transition_to(self, new_state: PatientState):
        """
        Move patient to a new state, recording history.
        """
        # Record the state we're leaving
        self.state_history.append((self.current_state, self.time_in_current_state))
        
        # Transition to new state
        self.current_state = new_state
        self.time_in_current_state = 0
    
    def advance_time(self, time_units: int = 1):
        """
        Progress time for this patient in their current state.
        """
        self.time_in_current_state += time_units
        self.total_time += time_units
    
    def is_terminal(self) -> bool:
        """
        Check if patient is in a terminal state (deceased or long-term remission).
        """
        return self.current_state in [PatientState.DECEASED, PatientState.REMISSION]


# ============================================================================
# TRANSITION RULES
# ============================================================================

@dataclass
class TransitionRule:
    """
    Defines when and how a patient moves from one state to another.
    """
    from_state: PatientState
    to_state: PatientState
    min_duration: int  # Minimum time before transition can occur
    probability: float  # Probability of transition per time unit after min_duration
    condition: Optional[Callable[[Patient], bool]] = None  # Optional custom condition
    
    def can_transition(self, patient: Patient) -> bool:
        """
        Check if this transition rule applies to the patient.
        """
        # Must be in the correct state
        if patient.current_state != self.from_state:
            return False
        
        # Must have spent minimum time in state
        if patient.time_in_current_state < self.min_duration:
            return False
        
        # Check custom condition if provided
        if self.condition and not self.condition(patient):
            return False
        
        # Apply probability
        return random.random() < self.probability
    

# ============================================================================
# FLOW MODEL
# ============================================================================

class TreatmentFlowModel:
    """
    Orchestrates patient treatment flows by defining transition rules
    and simulating patient journeys through states.
    """
    
    def __init__(self):
        self.patients: List[Patient] = []
        self.transition_rules: List[TransitionRule] = []
        self.current_time: int = 0
        
    def add_transition_rule(self, rule: TransitionRule):
        """Add a transition rule to the model"""
        self.transition_rules.append(rule)
    
    def add_patient(self, patient: Patient):
        """Add a patient to the simulation"""
        self.patients.append(patient)
    
    def step(self):
        """
        Advance simulation by one time unit.
        For each patient, check transitions and advance time.
        """
        self.current_time += 1
        
        for patient in self.patients:
            # Skip patients in terminal states
            if patient.is_terminal():
                continue
            
            # Check all transition rules to see if patient should change state
            transitioned = False
            for rule in self.transition_rules:
                if rule.can_transition(patient):
                    patient.transition_to(rule.to_state)
                    transitioned = True
                    break  # Only one transition per time step
            
            # Always advance time for the patient
            patient.advance_time(1)
    
    def run(self, max_time: int):
        """
        Run the simulation for a specified number of time units.
        """
        for _ in range(max_time):
            self.step()
    
    # ========================================================================
    # DATA READOUT METHODS
    # ========================================================================
    
    def get_state_distribution(self) -> Dict[PatientState, int]:
        """Get current count of patients in each state"""
        distribution = {state: 0 for state in PatientState}
        for patient in self.patients:
            distribution[patient.current_state] += 1
        return distribution
    
    def get_average_time_in_state(self, state: PatientState) -> float:
        """Calculate average time patients spend in a given state"""
        times = []
        for patient in self.patients:
            for hist_state, duration in patient.state_history:
                if hist_state == state:
                    times.append(duration)
        return sum(times) / len(times) if times else 0
    
    def get_patient_outcomes(self) -> Dict[str, int]:
        """Summarize final outcomes for all patients"""
        outcomes = {"deceased": 0, "remission": 0, "in_treatment": 0}
        for patient in self.patients:
            if patient.current_state == PatientState.DECEASED:
                outcomes["deceased"] += 1
            elif patient.current_state == PatientState.REMISSION:
                outcomes["remission"] += 1
            else:
                outcomes["in_treatment"] += 1
        return outcomes
    
    def get_patient_journeys(self) -> List[Dict]:
        """Get detailed journey information for each patient"""
        journeys = []
        for patient in self.patients:
            journey = {
                "patient_id": patient.id,
                "current_state": patient.current_state.value,
                "total_time": patient.total_time,
                "history": [(s.value, d) for s, d in patient.state_history]
            }
            journeys.append(journey)
        return journeys
    
    def print_summary(self):
        """Print a summary of the simulation results"""
        print(f"\n{'='*60}")
        print(f"SIMULATION SUMMARY (Time: {self.current_time})")
        print(f"{'='*60}")
        
        print(f"\nTotal Patients: {len(self.patients)}")
        
        print("\nCurrent State Distribution:")
        for state, count in self.get_state_distribution().items():
            print(f"  {state.value:20s}: {count:4d} patients")
        
        print("\nPatient Outcomes:")
        for outcome, count in self.get_patient_outcomes().items():
            print(f"  {outcome:20s}: {count:4d} patients")
        
        print("\nAverage Time in Each State:")
        for state in PatientState:
            avg_time = self.get_average_time_in_state(state)
            print(f"  {state.value:20s}: {avg_time:6.1f} time units")


# ============================================================================
# EXAMPLE USAGE
# ============================================================================

def create_example_model():
    """
    Create an example treatment flow model with realistic transitions.
    This demonstrates how to set up a simple treatment pathway.
    """
    model = TreatmentFlowModel()
    
    # Define transition rules for a typical treatment flow
    # Format: from_state -> to_state (min_duration, probability per time unit)
    
    # From diagnosis to first treatment (immediate, certain)
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.DIAGNOSED,
        to_state=PatientState.TREATMENT_1,
        min_duration=0,
        probability=1.0
    ))
    
    # Treatment 1 outcomes (after minimum 3 time units)
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_1,
        to_state=PatientState.TREATMENT_2,
        min_duration=3,
        probability=0.15  # 15% chance per time unit to progress
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_1,
        to_state=PatientState.DECEASED,
        min_duration=3,
        probability=0.02  # 2% mortality risk per time unit
    ))
    
    # Treatment 2 outcomes
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.TREATMENT_3,
        min_duration=4,
        probability=0.10
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.REMISSION,
        min_duration=4,
        probability=0.08  # 8% chance of remission per time unit
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_2,
        to_state=PatientState.DECEASED,
        min_duration=4,
        probability=0.03
    ))
    
    # Treatment 3 outcomes
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_3,
        to_state=PatientState.REMISSION,
        min_duration=5,
        probability=0.12
    ))
    
    model.add_transition_rule(TransitionRule(
        from_state=PatientState.TREATMENT_3,
        to_state=PatientState.DECEASED,
        min_duration=5,
        probability=0.05
    ))
    
    return model


if __name__ == "__main__":
    # Create the model
    model = create_example_model()
    
    # Add 100 patients to the simulation
    for i in range(100):
        model.add_patient(Patient(id=i))
    
    # Run simulation for 50 time units
    print("Running simulation...")
    model.run(max_time=50)
    
    # Display results
    model.print_summary()
    
    # Example: Get detailed journey for first patient
    print("\n" + "="*60)
    print("EXAMPLE: First Patient's Journey")
    print("="*60)
    journeys = model.get_patient_journeys()
    first_journey = journeys[0]
    print(f"Patient {first_journey['patient_id']}:")
    print(f"  Current State: {first_journey['current_state']}")
    print(f"  Total Time: {first_journey['total_time']}")
    print(f"  Journey:")
    for state, duration in first_journey['history']:
        print(f"    {state:20s} for {duration} time units")

Running simulation...

SIMULATION SUMMARY (Time: 50)

Total Patients: 100

Current State Distribution:
  diagnosed           :    0 patients
  treatment_1         :    0 patients
  treatment_2         :    0 patients
  treatment_3         :    1 patients
  remission           :   71 patients
  deceased            :   28 patients

Patient Outcomes:
  deceased            :   28 patients
  remission           :   71 patients
  in_treatment        :    1 patients

Average Time in Each State:
  diagnosed           :    0.0 time units
  treatment_1         :    8.4 time units
  treatment_2         :    8.5 time units
  treatment_3         :   10.0 time units
  remission           :    0.0 time units
  deceased            :    0.0 time units

EXAMPLE: First Patient's Journey
Patient 0:
  Current State: remission
  Total Time: 9
  Journey:
    diagnosed            for 0 time units
    treatment_1          for 3 time units
    treatment_2          for 5 time units
